### importar librerias

In [67]:
import pandas as pd
import numpy as np
from bdatos import Bdatos
from ingestar import Ingestar

import warnings
warnings.filterwarnings("ignore")

### Inicializar

In [68]:
db_path = "/workspaces/piv_2025_2/src/proyecto_integrador/static/db/basedatos.db"
bd = Bdatos(db_path)
ingestar = Ingestar()

### ingestar informacion en un df

In [69]:
#blastchar/telco-customer-churn
df = pd.DataFrame()
dataset = ingestar.download_dataset_zip(kaggle_ref="blastchar/telco-customer-churn")
ruta_unzip= ingestar.extract_zip_files(dataset)
df = ingestar.load_dataset_as_dataframe(ruta_unzip)
df.head(3)

Descargando dataset desde Kaggle...
Ruta al dataset: /home/codespace/.cache/kagglehub/datasets/blastchar/telco-customer-churn/versions/1
Archivos encontrados en la descarga: ['WA_Fn-UseC_-Telco-Customer-Churn.csv']
Se detectaron archivos CSV directamente en la carpeta descargada.
Leyendo CSV /home/codespace/.cache/kagglehub/datasets/blastchar/telco-customer-churn/versions/1/WA_Fn-UseC_-Telco-Customer-Churn.csv ...
✅ Dataset cargado correctamente en DataFrame.


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


### Tablas, insertar y consultar informacion

In [20]:
bd.insertar_dataframe(df=df,tabla="tbl_churn")

In [26]:
df.to_csv("/workspaces/piv_2025_2/src/proyecto_integrador/static/csv/churn.csv")

In [29]:
query = "select count(*) from tbl_churn where gender = 'Male' "
df_2 = bd.consultar(query)
df_2.head(2)

,count(*)
0,3555


### Limpieza de datos

In [73]:
df_3 = df.copy()
df_3.to_csv("/workspaces/piv_2025_2/src/proyecto_integrador/static/csv/dataset_enriquecido.csv")

In [74]:
# Limpieza
df_3.drop_duplicates(inplace=True)

# Convertir TotalCharges a numérico (algunos valores pueden ser espacios vacíos)
df_3['TotalCharges'] = pd.to_numeric(df_3['TotalCharges'], errors='coerce')

# Manejo de nulos: eliminar filas con nulos en TotalCharges
df_3.dropna(subset=['TotalCharges'], inplace=True)

# Normalización de texto en categóricas
for col in df_3.select_dtypes(include='object').columns:
    df_3[col] = df_3[col].str.strip().str.lower()

In [75]:
df_3.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-vhveg,female,0,yes,no,1,no,no phone service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one year,no,mailed check,56.95,1889.50,no
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed check,53.85,108.15,yes


### Enriquecimiento de datos

In [76]:
# Definir rango de fechas
start_date = pd.to_datetime("2022-01-01")
end_date = pd.to_datetime("2024-12-31")

# Número de filas
n = len(df_3)

# Generar fechas aleatorias
random_days = np.random.randint(0, (end_date - start_date).days, n)
df_3['Fecha'] = start_date + pd.to_timedelta(random_days, unit='D')

# 4. Extraer año, mes y día
df_3['Año'] = df_3['Fecha'].dt.year
df_3['Mes'] = df_3['Fecha'].dt.month
df_3['Día'] = df_3['Fecha'].dt.day

# 5. Mostrar resultado
print(df_3[['Fecha','Año','Mes','Día']].head())

       Fecha   Año  Mes  Día
0 2022-04-21  2022    4   21
1 2022-06-11  2022    6   11
2 2022-10-09  2022   10    9
3 2023-06-15  2023    6   15
4 2023-03-01  2023    3    1


In [79]:
df.head(3)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-vhveg,female,0,yes,no,1,no,no phone service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one year,no,mailed check,56.95,1889.50,no
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed check,53.85,108.15,yes


In [80]:
df_3.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Fecha,Año,Mes,Día
0,7590-vhveg,female,0,yes,no,1,no,no phone service,dsl,no,...,month-to-month,yes,electronic check,29.85,29.85,no,2022-04-21,2022,4,21
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,one year,no,mailed check,56.95,1889.50,no,2022-06-11,2022,6,11
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,month-to-month,yes,mailed check,53.85,108.15,yes,2022-10-09,2022,10,9
